In [1]:
import requests
import numpy as np


FORECAST_URL = 'https://api.open-meteo.com/v1/forecast'

HISTORICAL_DATA_URL = 'https://archive-api.open-meteo.com/v1/era5'

FORECAST_PARAMS = {
    'latitude':39.9439,
    'longitude': 32.8560,
    'hourly': ['temperature_2m', 'relativehumidity_2m', 'dewpoint_2m', 'precipitation', 'rain', 'windspeed_10m', 'winddirection_10m', 'soil_moisture_9_27cm', 'cloudcover']
}

HISTORICAL_PARAMS = {
    'latitude':39.9439,
    'longitude': 32.8560,
    'hourly': ['temperature_2m', 'relativehumidity_2m', 'dewpoint_2m', 'precipitation', 'rain', 'windspeed_10m', 'soil_moisture_7_to_28cm', 'cloudcover']
}

forecast_r = requests.get(url = FORECAST_URL, params = FORECAST_PARAMS)

HISTORICAL_PARAMS['start_date'] = '2021-01-01'
HISTORICAL_PARAMS['end_date'] = '2022-01-01'

historical_r = requests.get(url = HISTORICAL_DATA_URL, params = HISTORICAL_PARAMS)

forecast_data = forecast_r.json()
historical_data = historical_r.json()



In [18]:
historical_last_month = {}

for i in historical_data['hourly']:
    historical_last_month[i] = historical_data['hourly'][i][:24*30]

print(len(historical_last_month['temperature_2m'])/24)

30.0


In [19]:
for i in forecast_data['hourly']:
    print(i, end=' ')
print('****************')
for i in historical_data['hourly']:
    print(i, end=' ')

# print('****************')
# print('Forecast Data for 2 meters temperature')

# for element in zip(forecast_data['hourly']['time'][:100], forecast_data['hourly']['temperature_2m'][:100], forecast_data['hourly']['relativehumidity_2m'][:100]
# , forecast_data['hourly']['rain'][:100]):
#     print(element)

# print('****************')
# print('Historical Data for 2 meters temperature')

for i, element in enumerate(zip(historical_data['hourly']['time'][:10], historical_data['hourly']['temperature_2m'][:10])):
    print(i, element)

print()
print(len(forecast_data['hourly']['temperature_2m']))
print(len(historical_data['hourly']['temperature_2m'])/24)

time temperature_2m relativehumidity_2m dewpoint_2m precipitation rain windspeed_10m winddirection_10m soil_moisture_9_27cm cloudcover ****************
time temperature_2m relativehumidity_2m dewpoint_2m precipitation rain windspeed_10m soil_moisture_7_to_28cm cloudcover 0 ('2021-01-01T00:00', 3.2)
1 ('2021-01-01T01:00', 3.4)
2 ('2021-01-01T02:00', 3.0)
3 ('2021-01-01T03:00', 2.3)
4 ('2021-01-01T04:00', 1.1)
5 ('2021-01-01T05:00', 0.3)
6 ('2021-01-01T06:00', 0.8)
7 ('2021-01-01T07:00', 4.4)
8 ('2021-01-01T08:00', 8.6)
9 ('2021-01-01T09:00', 10.6)

168
366.0


In [20]:
def get_pth_percentile(array, percentile):
    sorted = np.sort(array)

    ith = int((percentile/100)*(len(array)-1))

    return sorted[ith]


In [60]:
high_thresholds = {}
for i in historical_data['hourly']:
    high_thresholds[i] = get_pth_percentile(historical_data['hourly'][i], 90)

low_thresholds = {}
for i in historical_data['hourly']:
    low_thresholds[i] = get_pth_percentile(historical_data['hourly'][i], 10)

print(high_thresholds)
print(low_thresholds)

###########################################################################

high_thresholds = {}
for i in historical_last_month:
    high_thresholds[i] = get_pth_percentile(historical_last_month[i], 90)

low_thresholds = {}
for i in historical_last_month:
    low_thresholds[i] = get_pth_percentile(historical_last_month[i], 10)

print('\n', high_thresholds)
print(low_thresholds)

{'time': '2021-11-26T08:00', 'temperature_2m': 25.5, 'relativehumidity_2m': 90, 'dewpoint_2m': 11.4, 'precipitation': 0.0, 'rain': 0.0, 'windspeed_10m': 13.2, 'soil_moisture_7_to_28cm': 0.334, 'cloudcover': 96}
{'time': '2021-02-06T14:00', 'temperature_2m': 0.2, 'relativehumidity_2m': 28, 'dewpoint_2m': -4.4, 'precipitation': 0.0, 'rain': 0.0, 'windspeed_10m': 3.4, 'soil_moisture_7_to_28cm': 0.14, 'cloudcover': 0}

 {'time': '2021-01-27T23:00', 'temperature_2m': 10.7, 'relativehumidity_2m': 94, 'dewpoint_2m': 5.1, 'precipitation': 0.3, 'rain': 0.1, 'windspeed_10m': 13.3, 'soil_moisture_7_to_28cm': 0.331, 'cloudcover': 100}
{'time': '2021-01-03T23:00', 'temperature_2m': -3.9, 'relativehumidity_2m': 50, 'dewpoint_2m': -10.0, 'precipitation': 0.0, 'rain': 0.0, 'windspeed_10m': 3.3, 'soil_moisture_7_to_28cm': 0.194, 'cloudcover': 2}


### Increasing Irrigation

Temperature is high / humidity is low / wind speed is high

In [57]:
def increase_irrigation_alert(forecasted_data):
    alerts = {}
    alert_type = {}
    alert_type['temperature_2m'] = 'high'
    alert_type['relativehumidity_2m'] = 'low'
    alert_type['windspeed_10m'] = 'high'
    day = 0
    for i, row in enumerate(zip(forecasted_data['temperature_2m'], forecasted_data['relativehumidity_2m'], forecasted_data['windspeed_10m'])):
        if(i%24 == 0):
            day += 1
            alerts[day] = {}
            alerts[day]['temperature_2m'] = []
            alerts[day]['relativehumidity_2m'] = []
            alerts[day]['windspeed_10m'] = []
            
        if(row[0] > high_thresholds['temperature_2m']):
            alerts[day]['temperature_2m'].append(forecasted_data['time'][i])

        if(row[1] < low_thresholds['relativehumidity_2m']):
            alerts[day]['relativehumidity_2m'].append(forecasted_data['time'][i])

        if(row[2] > high_thresholds['windspeed_10m']):
            alerts[day]['windspeed_10m'].append(forecasted_data['time'][i])

    return alerts, alert_type

y = increase_irrigation_alert(forecast_data['hourly'])


### Decreasing Irrigation

Rain, precipitation is high / soil moisture is high

In [58]:
def decrease_irrigation_alert(forecasted_data):
    alerts = {}
    alert_type = {}
    alert_type['precipitation'] = 'high'
    alert_type['soil_moisture_9_27cm'] = 'high'
    day = 0
    for i, row in enumerate(zip(forecasted_data['precipitation'], forecasted_data['soil_moisture_9_27cm'])):
        if(i%24 == 0):
            day += 1
            alerts[day] = {}
            alerts[day]['precipitation'] = []
            alerts[day]['soil_moisture_9_27cm'] = []
        #     print(f'DAY {day}: ')
        # print(i, row)
        if(row[0] > high_thresholds['precipitation']):
            alerts[day]['precipitation'].append(forecasted_data['time'][i])

        if(row[1] > high_thresholds['soil_moisture_7_to_28cm']):
            alerts[day]['soil_moisture_9_27cm'].append(forecasted_data['time'][i])
    return alerts, alert_type

x = decrease_irrigation_alert(forecast_data['hourly'])

### Frost Warning

- A forecasted overnight low less than 40 degrees F. (4.4 C.). 
- A dewpoint lower than 40 degrees F. 
- A cloudless night. (Be aware skies can clear overnight.) 
- A windless night.

Read more at Gardening Know How: How To Predict Frost And Protect Your Garden https://www.gardeningknowhow.com/plant-problems/environmental/how-to-predict-frost.htm

In [70]:
type(forecast_data['hourly']['temperature_2m'])
forecast_data['hourly']['dewpoint_2m'][:8]

[-0.5, -0.6, -0.7, -0.9, -1.4, -1.8, -0.7, -0.5]

In [78]:
def frost_warning(forecasted_data):
    
    for i in range(0, 7):
        start_index = i*24
        avg_degree = sum(forecasted_data['temperature_2m'][start_index:start_index+8])/8
        avg_dew = sum(forecasted_data['dewpoint_2m'][start_index:start_index+8])/8
        avg_wind = sum(forecasted_data['windspeed_10m'][start_index:start_index+8])/8
        avg_cloud = sum(forecasted_data['cloudcover'][start_index:start_index+8])/8

        temp_wind_threshold = 5
        temp_cloud_threshold = 50
        if(avg_degree < 4.4 and avg_dew < 4.4 and avg_wind < temp_wind_threshold and avg_cloud < temp_cloud_threshold):
            print(f'\n Day: {i} \n The frost is expected!!!!!!!')

frost_warning(forecast_data['hourly'])
print()
print(forecast_data['hourly']['temperature_2m'][24:32])
print(forecast_data['hourly']['dewpoint_2m'][24:32])
print(forecast_data['hourly']['windspeed_10m'][24:32])
print(forecast_data['hourly']['cloudcover'][24:32])


 Day: 2 
 The frost is expected!!!!!!!

 Day: 3 
 The frost is expected!!!!!!!

[-3.4, -3.6, -3.8, -3.7, -3.5, -3.3, -2.9, -2.0]
[-4.0, -4.2, -4.3, -4.2, -4.0, -3.7, -3.4, -2.7]
[3.1, 2.9, 1.3, 1.1, 1.1, 1.5, 1.5, 1.0]
[23, 38, 59, 96, 95, 100, 100, 100]


### Recommendation of Irrigation Schedule

May be determined according to the sunrise/sunset

## Recommending Based On Alert Messages

In [59]:
week_dict = {1: 'Monday', 2: 'Tuesday', 3: 'Wednesday', 4: 'Thursday', 5: 'Friday', 6: 'Saturday', 7: 'Sunday'}

def hour(time_string):
    return int(time_string[-5:-3])

def investigate_one_day(day_message):
    if(len(day_message) == 0): return []
    message_intervals = []

    i = 0
    current_first_end = day_message[i]
    while(True):
        current_last_end = day_message[i]
        if(i >= len(day_message)-1):
            message_intervals.append((current_first_end, current_last_end))
            break
        if(hour(day_message[i]) + 1 != hour(day_message[i+1])):
            message_intervals.append((current_first_end, current_last_end))
            current_first_end = day_message[i+1]
        i += 1

    return message_intervals

def recommend(alert_message, alert_message_types, given_message = 'You may consider decreasing irrigation', today = 1):
    for i in range(1, 8):
        current_day = week_dict[(today+i-1)%7+1]
        print(f'\nNotifications for {current_day}:')
        for type in alert_message[i]:
            if(alert_message[i][type] != []):
                print(f'In the following hours the {type} is {alert_message_types[type]}:')
                intervals = investigate_one_day(alert_message[i][type])
                for interval in intervals:
                    print(f'between {interval[0]} - {interval[1]}')
                print(given_message)

    pass

recommend(x[0], x[1], given_message = 'DECREASE SULAMAA AQKKKK')
print('\n##############################################')
recommend(y[0], y[1], given_message = 'ARTTIR KARDES BIRAZ OLDUK AK')


Notifications for Tuesday:
In the following hours the soil_moisture_9_27cm is high:
between 2023-01-07T00:00 - 2023-01-07T23:00
DECREASE SULAMAA AQKKKK

Notifications for Wednesday:
In the following hours the soil_moisture_9_27cm is high:
between 2023-01-08T00:00 - 2023-01-08T23:00
DECREASE SULAMAA AQKKKK

Notifications for Thursday:
In the following hours the soil_moisture_9_27cm is high:
between 2023-01-09T00:00 - 2023-01-09T23:00
DECREASE SULAMAA AQKKKK

Notifications for Friday:
In the following hours the soil_moisture_9_27cm is high:
between 2023-01-10T00:00 - 2023-01-10T23:00
DECREASE SULAMAA AQKKKK

Notifications for Saturday:
In the following hours the precipitation is high:
between 2023-01-11T04:00 - 2023-01-11T12:00
between 2023-01-11T16:00 - 2023-01-11T21:00
DECREASE SULAMAA AQKKKK
In the following hours the soil_moisture_9_27cm is high:
between 2023-01-11T00:00 - 2023-01-11T23:00
DECREASE SULAMAA AQKKKK

Notifications for Sunday:
In the following hours the precipitation is